In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('data/transcript_files'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from transformers import AutoModelForCausalLM,AutoTokenizer,TrainingArguments,Trainer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

/Users/aryamansriram/micromamba/envs/llama_fun/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 762/762 [00:00<00:00, 299kB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 7.95MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 50.9MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 11.6MB/s]


In [6]:
from datasets import load_dataset
data = load_dataset("text",data_dir="../src/data/transcript_files/")


Generating train split: 56 examples [00:00, 2869.16 examples/s]


In [7]:
# def get_sentences(examples):
#     concatenatedsum(examples['input_ids'], [])
#     result = {
#         'text': []
#     }

def preprocess_text(example):
    return tokenizer(example['text'])

tokenized_data = data.map(preprocess_text,remove_columns=data['train'].column_names)

Map: 100%|██████████| 56/56 [00:01<00:00, 38.82 examples/s]


In [9]:
block_size=512

def chunk_data(examples):
    #print(type(examples['input_ids']))
    
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    
    total_length = len(concatenated_examples['input_ids'])
    
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
   
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k,t in concatenated_examples.items()
    }
    result['labels'] = result['input_ids'].copy()
   
    return result
lm_dataset = tokenized_data.map(chunk_data,batched=True,remove_columns=tokenized_data['train'].column_names)

Map: 100%|██████████| 56/56 [00:00<00:00, 108.60 examples/s]


In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

model.safetensors: 100%|██████████| 353M/353M [00:06<00:00, 54.5MB/s] 
generation_config.json: 100%|██████████| 124/124 [00:00<00:00, 110kB/s]


In [14]:
training_args = TrainingArguments(
    output_dir="ad-test",
    evaluation_strategy="no",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none",
    do_eval=False, 
    num_train_epochs=1.0,
    logging_strategy='epoch'
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    data_collator=data_collator,
    tokenizer=tokenizer
    
)

trainer.train()

In [ ]:
prompt = 'How I think you can value a company like tesla'
input_dict = tokenizer(prompt,return_tensors='pt')
inputs = input_dict.input_ids
attn = input_dict.attention_mask
op = model.generate(inputs.cuda(),max_new_tokens=100, do_sample=True, top_k=10, top_p=0.95,attention_mask=attn.cuda())
tokenizer.batch_decode(op, skip_special_tokens=True)